# Lab2 - Expansão de Consultas
### Hadrizia Santos
Nesta atividade será exercitada a noção de expansão de consultas. Considerando a coleção de notícias do lab passado, deve-se executar os seguintes passos:

1. Escrever uma função que receba uma coleção de documentos e retorne uma matrix de termos-termos contendo as frequências de co-ocorrência de duas palavras consecutivas no texto (bigramas).
2. Escrever uma função que receba um certo termo de consulta e a matriz construída no passo 1 acima e retorneas top-3 palavras em ordem decrescente de frequencia.
3. Expandir a consulta original com os termos retornados no passo 2 acima.
4. Fazer uma busca disjuntiva (OR) considerando a nova consulta.

E responder às perguntas:

* Quais os termos retornados para a expansão de cada consulta?
* Você acha que esses termos são de fato relacionados com a consulta original? Justifique.
* Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?
* A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

## Importar bibliotecas e os dados necessários


In [146]:
import pandas as pd
import numpy as np
from scipy import sparse
import nltk
from nltk import bigrams   
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import scipy.sparse as sps
import math
import collections
import ast

from collections import OrderedDict

FILE_PATH = '../data/estadao_noticias_eleicao.csv'

df = pd.read_csv(FILE_PATH, encoding = 'utf-8')
df = df.replace(np.NAN, "")

# criação de uma nova coluna para a junção do título da notícia com seu conteúdo
df['noticia'] = df.titulo + ' ' + df.subTitulo + ' ' + df.conteudo

dictionary = collections.defaultdict(list)
idf_dict = {}
total_docs = len(df.noticia)

## 1. Construir matrix de ocorrência
**Obs:** O código abaixo que constrói a matriz de ocorrência foi copiado do repositório que está disponível em: https://github.com/allansales/information-retrieval/blob/master/Lab%202/coocurrence_matrix.ipynb

In [86]:
# O código abaixo está disponivel em: 
# https://github.com/allansales/information-retrieval/blob/master/Lab%202/coocurrence_matrix.ipynb

def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

def  generate_tokens_dataframe(noticia):
    # Removing punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    tokens_lists = noticia.apply(lambda text: tokenizer.tokenize(text.lower()))
    
    # Removing stopwords
    stopword_ = stopwords.words('portuguese')
    filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])
    
    tokens = [token for tokens_list in filtered_tokens for token in tokens_list]
    return tokens

def  generate_tokens_text(text):
    # Removing punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    tokens_lists = tokenizer.tokenize(text.lower())
    
    # Removing stopwords
    stopword_ = stopwords.words('portuguese')
    filtered_tokens = [token for token in tokens_lists if token not in stopword_]

    return filtered_tokens

def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

In [87]:
tokens = generate_tokens(df.noticia)
matrix, vocab = co_occurrence_matrix(tokens)
consultable_matrix = matrix.tocsr()
inverted_vocab = {vocab[key]: key for key in vocab}

## 2. Retornar as top-3 palavras mais frequentes de acordo com a matriz de ocorrência
Escrever uma função que receba uma palavra do dicionário e retorne quais as 3 palavras que ocorrem mais frequentemente com o input.

In [88]:
def get_co_ocurrences(co_matrix, inv_vocab, term, N=3):
    # sparse to dense representation
    np_array = np.reshape(co_matrix[term].toarray(), -1)
    # get indice of N occurences
    return list(OrderedDict({inv_vocab[idx]: np_array[idx] for idx in (-np_array).argsort()[:N]}).keys())

## 3. Expandir a consulta original com os termos retornados no passo 2 acima.
Escrever uma função que receba a consulta, pega as 3 palavras que aparecem com maior frequência junto com esse termo e retorne uma consulta expandida.

In [114]:
def expand_query(terms):
    query = []
    terms = terms.split(' ')
    if len(terms) > 1:
            for term in terms:
                query.append(term)
                query.extend(get_co_ocurrences(consultable_matrix, inverted_vocab, vocab[term], 3))
    else:
        query.append(terms[0])
        query.extend(get_co_ocurrences(consultable_matrix, inverted_vocab, vocab[terms[0]], 3))
        
    # removing duplicates terms
    return " ".join(str(x) for x in set(query))

## 4. Fazer uma busca disjuntiva (OR) considerando a nova consulta.
Escrever uma função que recebe uma consulta de N termos e retorna uma consula disjuntiva entre os termos. O modelo utilizado será o BM25, que foi implementado na atividade anterior.

In [128]:
def create_indexes(tokens, docId):
    for word in tokens:
        if word in dictionary: 
            if docId in dictionary[word]:
                dictionary[word][docId] += 1 
            else:
                dictionary[word][docId] = 1 
        else:
            dictionary[word] = {}
            dictionary[word][docId] = 1
            
def create_idf(word):
    idf = calculate_idf(word)
    idf_dict[word] = idf
    
def calculate_idf(word):
    M = total_docs
    k = len(dictionary[word].keys())
    idf = math.log((M + 1) / k)
    return idf

def calculate_bm25(tf, k=5):
    return (( k + 1) * tf) / ( tf + k)

def bm25(query, k):
    query = query.lower().split(' ')
    
    intersect_dict = {}
    if len(query) > 1:
        for word in query:   
            for docId in dictionary[word]:
                if docId in intersect_dict:
                    intersect_dict[docId] = intersect_dict[docId] + calculate_bm25(dictionary[word][docId], k) * idf_dict[word]
                else:
                    intersect_dict[docId] = calculate_bm25(dictionary[word][docId], k) * idf_dict[word]
    else:
        for docId in dictionary[query[0]]:
            intersect_dict[docId] = calculate_bm25(dictionary[query[0]][docId], k) * idf_dict[query[0]]
            
    rank = [(i, intersect_dict[i]) for i in sorted(intersect_dict, key=intersect_dict.get, reverse=True)]

    return [doc[0] for doc in rank[:3]]

In [ ]:
# Creating inverted index, TF and IDF
for index, row in df.iterrows():  
    tokens = generate_tokens_text(row.noticia)
    create_indexes(tokens, row.idNoticia)
for word in dictionary:
    create_idf(word)

In [129]:
def or_bm25_search(query):
    expanded_query = expand_query(query)
    return bm25(expanded_query, k=3)
        

## Respondendo às perguntas
### Quais os termos retornados para a expansão de cada consulta?
As consultas a serem testadas são as seguintes: 'corrupção', 'segundo turno' e 'empresa petrobrás'

In [144]:
queries = ['segundo turno', 'lava jato', 'projeto lei', 'compra voto', 'ministério público']
exp_queries = []
for query in queries:
    ext_query = expand_query(query)
    exp_queries.append(ext_query)
    print('Query original: %s' % query)
    print('Query expandida: %s' % ext_query)

Query original: segundo turno
Query expandida: mandato é eleição eleições lugar segundo turno
Query original: lava jato
Query expandida: é deflagrada lava porque lato jato polícia
Query original: projeto lei
Query expandida: político lei projeto anistia responsabilidade ficha poder
Query original: compra voto
Query expandida: pasadena votos distrital voto compra dilma refinaria presidente
Query original: ministério público
Query expandida: é saúde fazenda ministério estadual público federal


### Você acha que esses termos são de fato relacionados com a consulta original? Justifique.
Acredito que os termos estejam relacionados com a consulta original nas duas últimas consultas, pois quando se fala em segundo turno, é normalmente sobre eleições e todos os termos estão relacionados ao contexto. O mesmo ocorre para petrobrás, onde os demais termos se relacionam com a consulta. Já a primeira consulta expandida me surpreendeu um pouco, pois não esperava que petrobrás aparecesse tanto junto de corrupção. Outra coisa que observei é que o termo '**é**', que deveria ser considerada stopword, pois apareceu em todas as consultas.


### Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?

In [137]:
# importing csv 
gabarito = pd.read_csv('../data/gabarito.csv')

### A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

In [139]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [147]:
def obj_to_list(obj):
    matrix = []
    for list_obj in obj:
        x = ast.literal_eval(list_obj)
        matrix.append(x)
    return matrix 

print('Resultado MAP de acordo com o gabarito')
print ("Precisão gabarito e BM25: %f \n" % (mapk(obj_to_list(gabarito.bm25), bm25_results, k=5)))

Resultado MAP de acordo com o gabarito


NameError: name 'bm25_results' is not defined